In [1]:
# 이제 document chain을 사용해 볼거임
# document를 이용해 LLM에게 질문을 한다는 거

# 그전에 우선 off-the-shelf chain을 사용할 거
# 그 다음에 LangChain Expression Language(LCEL)로 만들거

# 강의 영상 촬영당시에는 memory 파트에서 해봤던 LLMChain은 legacy가 됨
# 그리고 LCEL을 사용할 것을 권장하는 중 -> 한눈에 보기 쉽기 때문에

# off-the-shelf chain은 더 높은 수준의 작업을 수행하기 위한 기본적인 컴포넌트의 어셈블리
# off-the-shelf chain으로 document chain을 생성하는 방식은 다양하게 있음 -> stuff, refine, map reduce, map re-rank

In [2]:
# from langchain.chat_models import ChatOpenAI # -> ChatOpenAI는 langchain_openai에서 import하기
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter

# from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings # -> OpenAIEmbeddings는 langchain_openai에서 import하기
from langchain.embeddings import CacheBackedEmbeddings
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

from langchain.vectorstores import Chroma, FAISS
from langchain.storage import LocalFileStore

from langchain.chains import RetrievalQA


llm = ChatOpenAI(
    model_name="gpt-3.5-turbo-1106",
    temperature=0.1,
)

cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)

loader = UnstructuredFileLoader("./files/chapter_one.docx")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()
cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

# vectorstore = Chroma.from_documents(docs, cached_embeddings)
vectorstore = FAISS.from_documents(docs, cached_embeddings)

In [3]:
# 1) stuff는 쿼리를 입력하면 그와 관계있는 document들을 찾고, 한번에 llm에 입력해서 prompt를 stuff(채우기) 하는데 사용한다는 거
# (query, docs1, docs2, docs3, ...)

chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="map_rerank",  # default = stuff
    retriever=vectorstore.as_retriever(),
    # retriever는 document를 많은 장소로부터 retrieve(선별하여 가져오기) 할 수 있게 하는 클래스의 인터페이스
    # 여기서 많은 장소는 vetor store 뿐만 아니라, cloud, DB 등을 말함
    # Retriever docs 내용
    # retriever는 구조화되지 않은 쿼리를 해석하여 document를 반환함 -> vetor store보다 일반적인 개념
    # retriever의 document store(문서 저장) 기능은 필수가 아님 -> 반환만 하면 됨
    # vector store를 retriever의 Backbone으로 사용할 수 있으나, 그렇지 않은 경우도 있음
)

In [ ]:
# chain.run("Where does Winston live?")

In [5]:
# chain.run("Describe Victory Mansions")

chain(
    {"query": "Describe Victory Mansions"}
)  # map_rerank에서는 template을 맞춰줘야 작동함

d:\FULLSTACK-GPT\env\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(
d:\FULLSTACK-GPT\env\Lib\site-packages\langchain\chains\llm.py:369: UserWarning: The apply_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


{'query': 'Describe Victory Mansions',
 'result': 'Victory Mansions is a building with a gritty hallway that smells of boiled cabbage and old rag mats. The building has a poster with a large face and is equipped with a telescreen that cannot be shut off completely.'}

In [ ]:
# 2) refine은 모든 document를 돌면서 답변의 정확도를 높이는 방법임 (정제)
# (query, docs1, docs2, docs3, ...)
# -> 하나의 document를 읽고 거기있는 정보로 답변을 업데이트해가는 것
# -> 비용 비쌈

# 3) map reduce는 document들을 입력받아서, 개별적으로 요약작업을 수행함 -> 그리고 요약결과들을 llm에 넘겨주고 답변을 생성
# (docs1), (docs2), (docs3), ...
# -> 원래는 하둡에서 많은 양의 데이터를 처리하는 방법임 -> 각각의 데이터(문서)를 일정크기로 mapping하고 그걸 축소(reduce)하는 작업
# -> 크고 많은 연산이 필요함 -> 비용 비쌈

# 4) map re-rank는 쿼리와 관련된 document들을 쌍을 지어 llm에 넘겨 답변을 생성하고, 각 답변에 점수를 매김
# (query, docs1), (query, docs2), (query, docs3), ...
# -> 최종적으로 가장 높은 점수를 획득한 답변과 그 점수를 함께 반환함

# RetrievalQA의 특징 중 하나는 chain_type을 쉽게 바꿀 수 있음 -> stuff, refine, map_reduce, map_rerank
# 근데 LCEL이 보편적이고 off-the-shelf chain는 legacy이기 때문에 나중에도 쓸 수 있을지는 모름